In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [78]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", model_max_length=4096)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [35]:
output = tokenizer("hallo", return_tensors="pt")	

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


torch.Size([1, 2, 250112])

In [82]:
# Sample input text
input_text = "Translate German to English: ja ja ja"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", padding='max_length', max_length=4096, truncation=True)

# Prepare decoder input ids (usually the start token)
decoder_input_ids = tokenizer("", return_tensors="pt", padding='max_length', max_length=4096, truncation=True).input_ids
decoder_input_attention_mask = tokenizer("", return_tensors="pt", padding='max_length', max_length=4096, truncation=True).attention_mask


# Forward pass with decoder input ids
outputs = model.base_model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_input_attention_mask)

# Extract the logits 
logits = outputs.logits
logits.shape

torch.Size([1, 4096, 250112])

In [75]:
inputs

{'input_ids': tensor([[89349, 20567,   288,  5413,   267,   432,   432,   432,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [63]:
total_params = 0
for p in model.base_model.parameters():
    total_params += p.numel()
total_params

300176768

In [ ]:
# B, L
#    L

In [65]:
crit = torch.nn.CrossEntropyLoss()

In [71]:
output = torch.stack((logits[:, -1, 36339], logits[:, -1, 375]), dim=1)
target = torch.tensor([1.0,0.0]).unsqueeze(0)


In [83]:
output.shape, target.shape

(torch.Size([1, 2]), torch.Size([1, 2]))

In [72]:
crit(output, target)

tensor(5.1451, grad_fn=<DivBackward1>)

In [84]:
logits[0, -1, 36339], logits[0, -1, 375]

(tensor(-38.5166, grad_fn=<SelectBackward0>),
 tensor(-37.3664, grad_fn=<SelectBackward0>))

In [36]:
model(**output)

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [40]:
import torch

In [ ]:
def compute_rank_loss(logits_pos, logits_neg):
    r_pos = torch.sigmoid(logits_pos)
    r_neg = torch.sigmoid(logits_neg)
    diff = torch.sigmoid(r_pos - r_neg)
    return torch.log(1e-8 + torch.exp(diff))

In [11]:
type(output)

transformers.tokenization_utils_base.BatchEncoding

In [16]:
import torch

In [18]:
output['input_ids'] = torch.tensor(output['input_ids'])
output['attention_mask'] = torch.tensor(output['attention_mask'])

/tmp/ipykernel_6469/3437213571.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output['input_ids'] = torch.tensor(output['input_ids'])


In [20]:
model.base_model(output, return_dict=True)

AttributeError: 

In [21]:
model.get_output_embeddings()

Linear(in_features=512, out_features=250112, bias=False)

In [ ]:
model.base_model

In [12]:
model.base_model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          